In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from quick_pp.utils import line_intersection
from quick_pp.fluid_type import fit_pressure_gradient, fluid_contact_plot, func

In [ ]:
pres = pd.read_csv(r"data\01_raw\formation_pressure_data.csv")

In [ ]:
plt.figure(figsize=(6, 7))
pres['HYDRO_STATIC_PRESSURE'] = pres['TVDSS'] * 0.433 + 14.7
fluid_contact_plot(pres['TVDSS'], pres['HYDRO_STATIC_PRESSURE'], 0.433, 14.7, 'HYDROSTATIC')
for fluid_type, data in pres.groupby('FLUID_TYPE'):
    m, c = fit_pressure_gradient(data['TVDSS'], data['FORMATION_PRESSURE'])
    fluid_contact_plot(data['TVDSS'], data['FORMATION_PRESSURE'], m, c, fluid_type,
                       ylim=(5300, 4600), xlim=(3400, 3600))
    print(m, c)


In [ ]:
line_constants = {
    'GAS': (0.085, 3031),
    'OIL': (0.342, 1760),
    'WATER': (0.431, 1315),
}
plt.figure(figsize=(6, 7))
pres['HYDRO_STATIC_PRESSURE'] = pres['TVDSS'] * 0.433 + 14.7
fluid_contact_plot(pres['TVDSS'], pres['HYDRO_STATIC_PRESSURE'], 0.433, 14.7, 'HYDROSTATIC')
for fluid_type, data in pres.groupby('FLUID_TYPE'):
    m = line_constants[fluid_type.upper()][0]
    c = line_constants[fluid_type.upper()][1]
    fluid_contact_plot(data['TVDSS'], data['FORMATION_PRESSURE'], m, c, fluid_type,
                       ylim=(5300, 4600), xlim=(3400, 3575))

# Identify fluid contacts
tvd_pts = np.linspace(pres.TVDSS.min(), pres.TVDSS.max(), 100)
gas_gradient_line = list(zip(func(tvd_pts, *line_constants['GAS']), tvd_pts))
oil_gradient_line = list(zip(func(tvd_pts, *line_constants['OIL']), tvd_pts))
water_gradient_line = list(zip(func(tvd_pts, *line_constants['WATER']), tvd_pts))

fp_goc, goc = line_intersection(
    (gas_gradient_line[1], gas_gradient_line[-1]),
    (oil_gradient_line[1], oil_gradient_line[-1])
)
fp_owc, owc = line_intersection(
    (oil_gradient_line[1], oil_gradient_line[-1]),
    (water_gradient_line[1], water_gradient_line[-1])
)

print(f'GOC: {goc:.2f} ft, FPGOC: {fp_goc:.2f} psi/ft')
print(f'OWC: {owc:.2f} ft, FPOWC: {fp_owc:.2f} psi/ft')